<a href="https://colab.research.google.com/github/mumukyung/Computervision_Seminar/blob/main/test_acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/CV_seminar_project

Mounted at /content/drive
/content/drive/MyDrive/CV_seminar_project


In [2]:
import torch
state_dict= torch.load('resnet50.pth')

In [3]:
batch_size = 8

In [4]:
# best_epoch = state_dic['epoch']
best_test_acc = state_dict['test_acc']
weights = state_dict['net']
print(f'최종적으로 {56}번째 epoch에서 validation셋 기준으로 {best_test_acc}를 달성하였습니다.')

최종적으로 56번째 epoch에서 validation셋 기준으로 93.52를 달성하였습니다.


2. 학습한 가중치를 모델에 적용하기

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정
torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)
device

'cuda'

In [6]:
from torchvision import models # 모델 라이브러리 함수

resnet_50 = models.resnet50(pretrained=True).to(device) # 선행학습 여부

# finetuning
import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
resnet_50.fc = nn.Linear(resnet_50.fc.in_features, 3).to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [7]:
# 학습한 가중치 적용 완료
resnet_50.load_state_dict(weights)

<All keys matched successfully>

3. test_set의 최종성능 확인하기

- homeworks
> 모델은 100 epoch를 돌려서 만든 가중치를 이용한다.   
> testloader를 생성하여, 최종 성능을 평가한다.   
> tensorbord는 자율로 이용하시오.

In [8]:
from dataset import Custom_dataset as C
from torch.utils.data import Dataset, DataLoader
import cv2
import os 
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 augmentation
import glob
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 # albumentations 텐서화 함수

root_path = '/content/drive/MyDrive/CV_seminar_project'

train_transforms = A.Compose([
    A.Resize(224,224),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.ChannelShuffle(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 이미지넷 데이터셋 통계값으로 Normalize
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 텐서타입은 안해줌
    ToTensorV2() # Normalize를 먼저하고 tensor화를 진행해야한다.
])

### Pytorch 데이터 클래스 생성

test_class = C(root_path=root_path, mode='test', transforms=test_transforms)

### Pytorch BatchLoader 생성 (학습에 이용할 최종 dataloader)
from torch.utils.data import DataLoader as DataLoader

test_loader = DataLoader(test_class, batch_size=batch_size, shuffle = False, num_workers=0)


In [9]:
criterion = nn.CrossEntropyLoss().to(device)
test_running_loss = 0.0

total = 0
correct = 0
test_acc = 0

resnet_50.eval()
with torch.no_grad():
  for ii, (test_img, test_label) in enumerate(test_loader):
        # gpu에 할당
    test_img = test_img.to(device)
    test_label = test_label.to(device)

    output = resnet_50(test_img) # 모델에 입력

    # acc
    _, predictions = torch.max(output.data ,dim = 1 )
    
    total += test_label.size(0)
    correct += (predictions == test_label).sum().item()
    test_acc += 100 * (correct/total)

result = test_acc/(ii+1)

testloader 생성하기

In [10]:
result

91.32830631774333